In [1]:
# !pip install ipython-autotime library to count run time
%load_ext autotime

time: 0 ns (started: 2021-09-19 23:40:08 +08:00)


In [1]:
import cv2
import os
from skimage.util import random_noise
from imwatermark import WatermarkEncoder
# from blind_watermark import WaterMark
from imwatermark import WatermarkDecoder
import numpy as np
import random
import glob
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
# this need to import seperately, else it will kernel die
from blind_watermark import WaterMark

bwm1 = WaterMark(password_img=1, password_wm=1)
bwm1.read_img('test/MRI_A.png')
wm = 'zhen'
bwm1.read_wm(wm, mode='str')

bwm1.embed('test/MRI_A.png')
len_wm = len(bwm1.wm_bit)
print('Put down the length of wm_bit {len_wm}'.format(len_wm=len_wm))

# Part A: Encode

In [ ]:
#Encode DWT A

infolder='in_img'
outfolder=os.path.join('out_img','algo_a')
in_img_list = os.listdir(infolder)
text='zhen'

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_A_'+img
    out_path = os.path.join(outfolder,out_name)
    
   
    encoder = WaterMark(password_img=1, password_wm=1)
    encoder.read_img(in_path)
    encoder.read_wm(text, mode='str')
    encoder.embed(out_path)
     
    print("Done Encoding: " + out_path)

In [ ]:
#Encode dwtDct B

infolder='in_img'
outfolder=os.path.join('out_img','algo_b')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()


for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_B_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDct')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


In [ ]:
#encode DwtDctSvd C

infolder='in_img'
outfolder=os.path.join('out_img','algo_c')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_C_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    
    img_encoded = encoder.encode(in_img,'dwtDctSvd')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


In [ ]:
#encode RivaGAN D

infolder='in_img'
outfolder=os.path.join('out_img','algo_d')
in_img_list = os.listdir(infolder)
text='zhen'

encoder = WatermarkEncoder()

for img in in_img_list:
    in_path = os.path.join(infolder,img)
    out_name='Algo_D_'+img
    out_path = os.path.join(outfolder,out_name)
    
    in_img = cv2.imread(in_path)
    
    encoder.set_watermark('bytes', text.encode('utf-8'))
    encoder.loadModel()
    img_encoded = encoder.encode(in_img,'rivaGan')
    
    cv2.imwrite(out_path,img_encoded)
    
    print("Done Encoding: " + out_path)
    


# Part B: Attacking

In [ ]:
#Attacking Methods

def gaussian(img, output_file_name):
    img_shape = img.shape
    gauss_img = random_noise(img, mode='gaussian', mean=0, var=0.01, clip= True)
    gauss_img = np.array(255*gauss_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, gauss_img)

def salt_pepper(img, output_file_name):
    ratio = 0.1 #Probability of the noise
    #0.01 all pass?
    #0.1 algorithm d only partial pass
    snp_img = np.zeros(img.shape,np.uint8)
    thres = 1 - ratio
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rdn = random.random()
            if rdn < ratio:
                snp_img[i][j] = 0
            elif rdn > thres:
                snp_img[i][j] = 255
            else:
                snp_img[i][j] = img[i][j]
    cv2.imwrite(output_file_name, snp_img)
    
def speckle(img, output_file_name):
    speckle_img = random_noise(img, mode='speckle', mean=0, var=0.01, clip= True)
    # The above function returns a floating-point image on the range [0, 1], thus we changed it to 'uint8' and from [0,255]
    speckle_img = np.array(255*speckle_img, dtype = 'uint8')
    cv2.imwrite(output_file_name, speckle_img)
    
def poisson(img, output_file_name):
    img_shape = img.shape
    noise = np.random.poisson(20, img_shape)
    #30 all fail 
    output = img + noise
    cv2.imwrite(output_file_name, output)
    
def rotate(img, output_file_name, angle=10):
    rows, cols, _ = img.shape
    M = cv2.getRotationMatrix2D(center=(cols / 2, rows / 2), angle=angle, scale=1)
    output_img = cv2.warpAffine(img, M, (cols, rows))
    cv2.imwrite(output_file_name, output_img)
    
def upscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=1.25, fy=1.25, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def downscale(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.resize(img, None, fx=0.75, fy=0.75, interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(output_file_name, output_img)
    
def averaging(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.blur(img,(5,5)) 
    cv2.imwrite(output_file_name, output_img)
    
def gaussian_blurring(img, output_file_name):
    img_shape = img.shape
    output_img = cv2.GaussianBlur(img,(5,5),0) 
    cv2.imwrite(output_file_name, output_img)
    
def median_blurring(img, output_file_name):
    output_img = cv2.medianBlur(img,7)
    #5 all pass
    cv2.imwrite(output_file_name, output_img)
    
def bilateral_filtering(img, output_file_name):
    output_img = cv2.bilateralFilter(img,9,75,75)
    cv2.imwrite(output_file_name, output_img)
    
def sharpen_filtering(img, output_file_name):
    sharpen_filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharped_img = cv2.filter2D(img, -1, sharpen_filter)
    cv2.imwrite(output_file_name, sharped_img)

def crop_horizontal(img, output_file_name):
    ratio = 0.5  
    #left 50% all pass
    img_shape = img.shape
    height = int(img_shape[0] * ratio)
    horizontal_img = img[:height, :, :]
    cv2.imwrite(output_file_name, horizontal_img)

def crop_vertical(img, output_file_name):
    ratio = 0.5
    img_shape = img.shape
    vertical = int(img_shape[1] * ratio)
    vertical_img = img[:, :vertical, :]
    cv2.imwrite(output_file_name, vertical_img)

def increase_brightness(img, output_file_name):
    ratio = 1.4 # 
    #+10% all pass
    #+50% algorithm D
    inc_bright_img = img * ratio
    inc_bright_img[inc_bright_img > 255] = 255
    cv2.imwrite(output_file_name, inc_bright_img)
    
def decrease_brightness(img, output_file_name):
    ratio = 0.6 
    # -50% brightness all pass
    dec_bright_img = img * ratio
    dec_bright_img[dec_bright_img > 255] = 255
    cv2.imwrite(output_file_name, dec_bright_img)
    
def masks(img, output_file_name):
    n = 5 # Set 2 mask
    ratio = 0.3 # mask ratio
    input_img_shape = img.shape
    mask_img = img.copy()
    for i in range(n):
        tmp = np.random.rand() * (1 - ratio)  # random one place to put mask，1-ratio to avoid overfloat
        start_height, end_height = int(tmp * input_img_shape[0]), int((tmp + ratio) * input_img_shape[0])
        tmp = np.random.rand() * (1 - ratio)
        start_width, end_width = int(tmp * input_img_shape[1]), int((tmp + ratio) * input_img_shape[1])

        mask_img[start_height:end_height, start_width:end_width, :] = 0
    cv2.imwrite(output_file_name, mask_img)
    
def init_attack(input_image, image_name):
    print('Attacking ' + image_name)
    
    a = image_name.split('.')[0]
    mri_name = a.split('_')[2]+'_'+a.split('_')[3]
    algo_name = a.split('_')[0]+'_'+a.split('_')[1]
    
    
    output_path=os.path.join('out_att',mri_name,algo_name.lower())
#     print(output_path)
    
    gaussian(input_image, os.path.join(output_path,'Gaussian_Noise.png'))
    salt_pepper(input_image, os.path.join(output_path,'Salt_Pepper.png'))
    speckle(input_image, os.path.join(output_path,'Speckle.png'))
    poisson(input_image, os.path.join(output_path,'Poisson.png'))
    rotate(input_image, os.path.join(output_path,'Rotate.png'), angle=10)
    upscale(input_image, os.path.join(output_path,'ScaleUp.png'))
    downscale(input_image, os.path.join(output_path,'ScaleDown.png'))
    averaging(input_image, os.path.join(output_path,'Averaging.png'))
    gaussian_blurring(input_image, os.path.join(output_path,'Gaussian_Blurring.png'))
    median_blurring(input_image, os.path.join(output_path,'Median_Blurring.png'))
    bilateral_filtering(input_image, os.path.join(output_path,'Bilateral_Filtering.png'))
    sharpen_filtering(input_image, os.path.join(output_path,'Sharpen_Filtering.png'))
    crop_horizontal(input_image, os.path.join(output_path,'Crop_Horizontal.png'))
    crop_vertical(input_image, os.path.join(output_path,'Crop_Vertical.png'))
    increase_brightness(input_image, os.path.join(output_path,'Brightness_Increase.png'))
    decrease_brightness(input_image, os.path.join(output_path,'Brightness_Decrease.png'))
    masks(input_image, os.path.join(output_path,'Masks.png'))
    cv2.imwrite(os.path.join(output_path,'JPG.jpg'), input_image)


In [ ]:
infolder=os.path.join('out_img')
outfolder=os.path.join('out_att')

img_folder_list = os.listdir(infolder) 

for folder in img_folder_list: 
#     print(folder)
    in_img_list = os.listdir(os.path.join(infolder,folder)) 
    
    for img_path in in_img_list: 
#         print(img_path)
        img = cv2.imread(os.path.join(infolder,folder,img_path))
        init_attack(img,img_path)   
        

# Part C: Decode

In [ ]:
def algo_A(image_list, result):
    count = 1
    print('ALGORITHM A')
    print('===========')
    for img in image_list:
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            bwm1=WaterMark(password_img=1,password_wm=1)
            output = bwm1.extract(img, wm_shape=31, mode='str')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')

    
def algo_B(image_list, result):
    count = 1
    print('ALGORITHM B')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            watermark = decoder.decode(bgr, 'dwtDct')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
def algo_C(image_list, result):
    count = 1
    print('ALGORITHM C')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            watermark = decoder.decode(bgr, 'dwtDctSvd')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
def algo_D(image_list, result):
    count = 1
    print('ALGORITHM D')
    print('===========')
    for img in image_list:
        bgr = cv2.imread(img)
        name = os.path.basename(img)
        name = name.split('.')[0]
        name = name.replace('_','')
        print(str(count) + ': ' + name)

        try:
            decoder = WatermarkDecoder('bytes', 32)
            decoder.loadModel()
            watermark = decoder.decode(bgr, 'rivaGan')
            output = watermark.decode('utf-8')
            print('Decoded text: ' + output)
            if output == 'zhen':
                print('Result      : Pass')
                result.append('Pass')
            else:
                print('Result      : Fail')
                result.append('Fail')
        except:
            print('Decoded text: Error')
            print('Result      : Fail')
            result.append('Fail')
            pass
        
        count += 1
        print('-----')
        
              
                


In [ ]:
def Decoder(df):
    img_folder_list = os.listdir('out_att') # Check all folder in this dic

    for folder in img_folder_list: 
        algo_list = os.listdir(os.path.join('out_att',folder)) # Check All algo folder
        
        for algo in algo_list: 
            imdir = os.path.join('out_att',folder,algo,'')
            ext = ['png', 'jpg'] 
            result = []
            result.append(folder)
            files = []
            [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
            image_list = [(file) for file in files]

            if algo == 'algo_a':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_A(image_list, result)
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_b':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_B(image_list, result)            
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_c':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_C(image_list, result)  
                print(result)
                df_length = len(df)
                df.loc[df_length] = result
                
            elif algo == 'algo_d':
                result.append(algo)
                print('[Photo: ' + folder + ']') # Show photo
                algo_D(image_list, result) 
                print(result)
                df_length = len(df)
                df.loc[df_length] = result  

In [ ]:
#initiate decode

title = ['Photo', 'Algorithm', 'Averaging', 'BilateralFiltering', 'BrightnessDecrease',
        'BrightnessIncrease', 'CropHorizontal', 'CropVertical', 'GaussianBlurring', 
        'GaussianNoise', 'Masks', 'MedianBlurring', 'Poisson', 'Rotate', 'SaltPeper',
        'ScaleDown', 'ScaleUp', 'SharpenFiltering', 'Speckle','JPG']

df = pd.DataFrame(columns=title)

Decoder(df)

output_path=os.path.join('csv','result2.csv')

df.to_csv(output_path, index = False)

In [ ]:
df

# Visualization

In [2]:
csv_path = os.path.join('csv', 'result2_sorted.csv')
df = pd.read_csv(csv_path)
df

,Photo,Algorithm,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
0,MRI_A,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
1,MRI_B,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Fail,Pass
2,MRI_C,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
3,MRI_D,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
4,MRI_E,algo_a,Pass,Pass,Pass,Fail,Pass,Fail,Pass,Fail,Pass,Pass,Pass,Fail,Pass,Fail,Fail,Fail,Pass,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MRI_P,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Fail,Pass,Pass,Pass,Pass
76,MRI_Q,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Pass,Pass,Pass,Pass,Pass
77,MRI_R,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Pass,Pass,Pass,Pass,Pass
78,MRI_S,algo_d,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Pass,Fail,Fail,Pass,Pass,Pass,Pass,Pass


In [3]:
# Prepocessing
# Define all attack column
att_column = df.iloc[:,2:20].columns

# Conver all att coloumn into integer
for x in range(len(att_column)):
    df[att_column[x]] = df[att_column[x]].map({'Pass': 1, 'Fail': 0})

df

,Photo,Algorithm,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
0,MRI_A,algo_a,1,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,1
1,MRI_B,algo_a,1,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1
2,MRI_C,algo_a,1,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,1
3,MRI_D,algo_a,1,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,1
4,MRI_E,algo_a,1,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,MRI_P,algo_d,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1
76,MRI_Q,algo_d,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1
77,MRI_R,algo_d,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1
78,MRI_S,algo_d,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1


In [4]:
df = df.drop(columns = ['Photo'])
df_sum = df.groupby(['Algorithm']).sum()
df_sum

,Averaging,BilateralFiltering,BrightnessDecrease,BrightnessIncrease,CropHorizontal,CropVertical,GaussianBlurring,GaussianNoise,Masks,MedianBlurring,Poisson,Rotate,SaltPeper,ScaleDown,ScaleUp,SharpenFiltering,Speckle,JPG
Algorithm,,,,,,,,,,,,,,,,,,
algo_a,20,20,20,0,20,0,20,0,20,20,20,0,20,0,0,0,19,20
algo_b,0,0,20,20,20,0,0,0,20,0,20,0,20,0,0,0,20,0
algo_c,20,9,20,20,20,0,20,1,20,19,20,0,20,0,0,0,20,20
algo_d,20,20,20,20,20,20,20,17,20,20,20,5,1,15,20,20,20,20


In [ ]:
s = df_sum
p1 = s.plot(kind='bar', stacked=True, figsize=(20, 10), title='Algorithm_Evaluation', xlabel='Algorithm Type', ylabel='No of Passed Cases')
p1.legend(loc='upper center')

In [ ]:
df_sum = df.groupby(['Algorithm']).sum()
df_sum_t = df_sum.transpose()
s = df_sum_t
p1 = s.plot(kind='bar', figsize=(20, 10), title='Attack_Evaluation', xlabel='Attack Method', ylabel='No of Passed Cases')
p1.yaxis.get_major_locator().set_params(integer=True)

# PSNR

In [ ]:
import os
import cv2
import pandas as pd
from math import log10, sqrt
import numpy as np

In [ ]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

### ORI VS ENCODED

In [ ]:
folder_1 = os.path.join('in_img')
folder_2 = os.path.join('out_img')

original_img = os.listdir(folder_1)
encoded_img = os.listdir(folder_2)

In [ ]:
list_1 = []
for item in original_img:
    img1 = cv2.imread(os.path.join(folder_1,item))
    list_1.append(img1)
   
list_2 = []
for folder in encoded_img: 
    img_list = os.listdir(os.path.join(folder_2,folder)) 
    
    for img_path in img_list: 
        img2 = cv2.imread(os.path.join(folder_2,folder,img_path))
        list_2.append(img2)
        
encoded_A = list_2[0:20]
encoded_B = list_2[20:40]
encoded_C = list_2[40:60]
encoded_D = list_2[60:80]

In [ ]:
length = len(list_1)

list_M1_1 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_A[x]
    psnr_score_m1_1 = PSNR(img1, img2)
    list_M1_1.append(psnr_score_m1_1)
    
list_M1_2 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_B[x]
    psnr_score_m1_2 = PSNR(img1, img2)
    list_M1_2.append(psnr_score_m1_2)
    
list_M1_3 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_C[x]
    psnr_score_m1_3 = PSNR(img1, img2)
    list_M1_3.append(psnr_score_m1_3)
    
list_M1_4 = []
for x in range(length):
    img1 = list_1[x]
    img2 = encoded_D[x]
    psnr_score_m1_4 = PSNR(img1, img2)
    list_M1_4.append(psnr_score_m1_4)

In [ ]:
df_M1 = pd.DataFrame(index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                            'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                            'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                            'MRI_S', 'MRI_T'])
df_M1['Encoded Algo A'] = list_M1_1
df_M1['Encoded Algo B'] = list_M1_2
df_M1['Encoded Algo C'] = list_M1_3
df_M1['Encoded Algo D'] = list_M1_4

df_M1

### ENCODED VS ATTACKED 

In [ ]:
folder_3 = os.path.join('out_img')
folder_4 = os.path.join('out_att')

encoded_img = os.listdir(folder_3)
attacked_img = os.listdir(folder_4)

In [ ]:
list_3 = []
for folder in encoded_img: 
    img_list = os.listdir(os.path.join(folder_3,folder)) 
    
    for img_path in img_list: 
        img3 = cv2.imread(os.path.join(folder_3,folder,img_path))
        list_3.append(img3)
        
encoded_A = list_3[0:20]
encoded_B = list_3[20:40]
encoded_C = list_3[40:60]
encoded_D = list_3[60:80]

list_4 = []
for folder in attacked_img: 
    img_list = os.listdir(os.path.join(folder_4,folder)) 
    
    for infolder in img_list:
        in_list = os.listdir(os.path.join(folder_4,folder,infolder))
    
        for img_path in in_list: 
            img4 = cv2.imread(os.path.join(folder_4,folder,infolder,img_path))
            list_4.append(img4)

### Algo A

In [ ]:
#algorithm A

Algorithm_att_a = []
a_psnr_list = []
x=0

for y in encoded_A: #20 combination
    start=x+(x*71)
    end=18+(x*72)
#     print(start,end)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            a_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            a_psnr_list.append('NaN')
    x=x+1

In [ ]:
n = 20
list_M2_1 = []
len_1 = len(a_psnr_list)
for i in range(n):
    start = int(i*len_1/n)
    end = int((i+1)*len_1/n)
    list_M2_1.append(a_psnr_list[start:end])

In [ ]:
df_M2_A = pd.DataFrame(list_M2_1, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_A

### Algo B

In [ ]:
#algorithm B
    
Algorithm_att_b = []
b_psnr_list = []
x=0

for y in encoded_B: #20 combination
    start=18+(x*72)
    end=36+(x*72)
#     print(start,end)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            b_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            b_psnr_list.append('NaN')
    x=x+1

In [ ]:
n = 20
list_M2_2 = []
len_2 = len(b_psnr_list)
for i in range(n):
    start = int(i*len_2/n)
    end = int((i+1)*len_2/n)
    list_M2_2.append(b_psnr_list[start:end])

In [ ]:
df_M2_B = pd.DataFrame(list_M2_2, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_B

### Algo C

In [ ]:
#algorithm C
    
Algorithm_att_c = []
c_psnr_list = []
x=0

for y in encoded_C: #20 combination
    start=36+(x*72)
    end=54+(x*72)
#     print(start,end)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            c_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            c_psnr_list.append('NaN')
    x=x+1

In [ ]:
n = 20
list_M2_3 = []
len_3 = len(c_psnr_list)
for i in range(n):
    start = int(i*len_3/n)
    end = int((i+1)*len_3/n)
    list_M2_3.append(c_psnr_list[start:end])

In [ ]:
df_M2_C = pd.DataFrame(list_M2_3, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_C

### Algo D

In [ ]:
#algorithm D

Algorithm_att_d = []
d_psnr_list = []
x=0

for y in encoded_D: #20 combination
    start=54+(x*72)
    end=72+(x*72)
    llist = list_4[start:end]    
    
    for z in llist: #18 combination
        try:
            d_psnr_list.append(PSNR(y,z)) # 20*18
        except:
            d_psnr_list.append('NaN')
    x=x+1

In [ ]:
n = 20
list_M2_4 = []
len_4 = len(d_psnr_list)
for i in range(n):
    start = int(i*len_4/n)
    end = int((i+1)*len_4/n)
    list_M2_4.append(d_psnr_list[start:end])

In [ ]:
df_M2_D = pd.DataFrame(list_M2_4, index=['MRI_A', 'MRI_B', 'MRI_C', 'MRI_D', 'MRI_E', 'MRI_F', 
                                          'MRI_G', 'MRI_H', 'MRI_I', 'MRI_J', 'MRI_K', 'MRI_L', 
                                          'MRI_M', 'MRI_N', 'MRI_O', 'MRI_P', 'MRI_Q', 'MRI_R', 
                                          'MRI_S', 'MRI_T'],
                                  columns=['Averaging','BilateralFiltering','BrightnessDecrease',
                                          'BrightnessIncrease','CropHorizontal','CropVertical',
                                          'GaussianBlurring','GaussianNoise','Masks','MedianBlurring',
                                          'Poisson','Rotate','SaltPeper','ScaleDown','ScaleUp',
                                          'SharpenFiltering','Speckle','JPG'])

df_M2_D